In [1]:
# import libraries

import pandas as pd
from pandas import set_option
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn import model_selection
import scipy as sy
from sklearn.preprocessing import OneHotEncoder
from sklearn import neighbors

from sklearn.feature_selection import RFE, f_regression
from sklearn.linear_model import (LinearRegression, Ridge, Lasso, RandomizedLasso)

In [2]:
# read train_data files
data_2016 = pd.read_csv("C:/Users/Deepika Keswarap/Documents/projecta/all/imputation/data2016.csv")
# read train_data file
data_2017 = pd.read_csv("C:/Users/Deepika Keswarap/Documents/projecta/all/imputation/data2017.csv")

In [3]:
# age
data_2016['age'] = 2018 - data_2016['yearbuilt']
data_2017['age'] = 2018 - data_2017['yearbuilt']

In [4]:
# Proportion of living area
data_2016['livingareaproportion'] = data_2016['calculatedfinishedsquarefeet']/data_2016['lotsizesquarefeet']
data_2017['livingareaproportion'] = data_2017['calculatedfinishedsquarefeet']/data_2017['lotsizesquarefeet']

In [5]:
# Total extra space
data_2016['extraspace'] = data_2016['lotsizesquarefeet']-data_2016['calculatedfinishedsquarefeet']
data_2017['extraspace'] = data_2017['lotsizesquarefeet']- data_2017['calculatedfinishedsquarefeet']

In [6]:
# bed:bath ratio
data_2016['bedbathratio'] = data_2016['bedroomcnt']/data_2016['bathroomcnt']
data_2017['bedbathratio'] = data_2017['bedroomcnt']/ data_2017['bathroomcnt']

In [7]:
#Average room size
data_2016['avgroomsize'] = data_2016['calculatedfinishedsquarefeet']/data_2016['roomcnt']
data_2017['avgroomsize'] = data_2017['calculatedfinishedsquarefeet']/data_2017['roomcnt']

In [8]:
# amenities score
#Garage - 20
#Pool- 25
#Hottub- 50
#AC- 10 
data_2016['garagescore'] = np.where(data_2016['garagecarcnt']>0, 20, 0)
data_2016['poolsscore'] = np.where(data_2016['poolcnt']>0, 25, 0)
data_2016['spascore'] = np.where(data_2016['hashottuborspa']>0, 50, 0)
data_2016['acscore'] = np.where(data_2016['airconditioningtypeid']!=5, 10, 0)                                                                          
data_2016['amenitiesscore'] = data_2016['garagescore'] + data_2016['poolsscore'] + data_2016['spascore'] + data_2016['acscore']


In [9]:
data_2017['garagescore'] = np.where(data_2017['garagecarcnt']>0, 20, 0)
data_2017['poolsscore'] = np.where(data_2017['poolcnt']>0, 25, 0)
data_2017['spascore'] = np.where(data_2017['hashottuborspa']>0, 50, 0)
data_2017['acscore'] = np.where(data_2017['airconditioningtypeid']!=5, 10, 0)                                                                          
data_2017['amenitiesscore'] = data_2017['garagescore'] + data_2017['poolsscore'] + data_2017['spascore'] + data_2017['acscore']

In [10]:
data_2016=data_2016.drop(['garagescore','poolsscore','spascore','acscore'], axis=1)


In [11]:
data_2017 =data_2017.drop(['garagescore','poolsscore','spascore','acscore'], axis=1)

In [12]:
data_2016.columns

Index(['logerror', 'airconditioningtypeid', 'basementsqft', 'bathroomcnt',
       'bedroomcnt', 'buildingqualitytypeid', 'calculatedbathnbr',
       'decktypeid', 'calculatedfinishedsquarefeet', 'finishedsquarefeet15',
       'finishedsquarefeet50', 'fips', 'fireplacecnt', 'garagecarcnt',
       'garagetotalsqft', 'hashottuborspa', 'heatingorsystemtypeid',
       'latitude', 'longitude', 'lotsizesquarefeet', 'poolcnt', 'poolsizesum',
       'pooltypeid2', 'pooltypeid7', 'propertylandusetypeid',
       'rawcensustractandblock', 'regionidcounty', 'regionidzip', 'roomcnt',
       'unitcnt', 'yardbuildingsqft17', 'yardbuildingsqft26', 'yearbuilt',
       'numberofstories', 'fireplaceflag', 'structuretaxvaluedollarcnt',
       'taxvaluedollarcnt', 'assessmentyear', 'landtaxvaluedollarcnt',
       'taxdelinquencyflag', 'taxpercentage', 'transaction_quarter', 'age',
       'livingareaproportion', 'extraspace', 'bedbathratio', 'avgroomsize',
       'amenitiesscore'],
      dtype='object')

In [13]:
#Number of properties in the zip
zip_count = data_2016['regionidzip'].value_counts().to_dict()
data_2016['N-zip_count'] = data_2016['regionidzip'].map(zip_count)


#Number of properties in the zip
zip_count = data_2017['regionidzip'].value_counts().to_dict()
data_2017['N-zip_count'] = data_2017['regionidzip'].map(zip_count)



In [14]:
data_2016=data_2016.drop(['pooltypeid2','pooltypeid7','yearbuilt','assessmentyear'], axis=1)

In [15]:
data_2017=data_2017.drop(['pooltypeid2','pooltypeid7','yearbuilt','assessmentyear'], axis=1)

In [16]:
# check missing values
data_2016[data_2016.isnull().any(axis=1)]

,logerror,airconditioningtypeid,basementsqft,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,decktypeid,calculatedfinishedsquarefeet,finishedsquarefeet15,...,taxdelinquencyflag,taxpercentage,transaction_quarter,age,livingareaproportion,extraspace,bedbathratio,avgroomsize,amenitiesscore,N-zip_count
126,-0.0020,5.0,0.0,0.0,0.0,7.0,2.0,0.0,1773.1923,1773.1923,...,0,0.011015,1,63.0,0.130564,11807.8077,NaN,inf,0,498
174,-0.0243,5.0,0.0,0.0,0.0,7.0,2.0,0.0,1773.1923,1773.1923,...,0,0.011474,1,63.0,0.060912,27337.3157,NaN,inf,0,110
231,-0.0545,5.0,0.0,0.0,0.0,7.0,2.0,0.0,1773.1923,1773.1923,...,0,0.026823,1,63.0,0.060912,27337.3157,NaN,inf,0,378
293,1.0830,5.0,0.0,0.0,0.0,7.0,2.0,0.0,1773.1923,1773.1923,...,0,0.013452,1,63.0,0.095859,16724.8077,NaN,inf,0,73
299,0.0611,5.0,0.0,0.0,0.0,7.0,2.0,0.0,1773.1923,1773.1923,...,0,0.013429,1,63.0,0.277148,4624.8077,NaN,inf,0,63
303,-0.4277,5.0,0.0,0.0,0.0,7.0,2.0,0.0,1773.1923,1773.1923,...,0,0.011927,1,63.0,0.090580,17802.8077,NaN,inf,0,250
313,-0.2971,5.0,0.0,0.0,0.0,7.0,2.0,0.0,896.0000,896.0000,...,0,0.010713,1,34.0,0.021652,40486.0000,NaN,inf,0,395
382,-0.2549,5.0,0.0,0.0,0.0,7.0,2.0,0.0,1773.1923,1773.1923,...,0,0.012415,1,63.0,0.060912,27337.3157,NaN,inf,0,74
387,0.0159,5.0,0.0,0.0,0.0,7.0,2.0,0.0,1773.1923,1773.1923,...,0,0.011094,1,63.0,0.284896,4450.8077,NaN,inf,0,347
727,0.1527,5.0,0.0,0.0,0.0,7.0,2.0,0.0,1392.0000,1392.0000,...,0,0.010667,1,34.0,0.343704,2658.0000,NaN,inf,0,245


In [17]:
data_2016=data_2016.fillna(0)
data_2017=data_2017.fillna(0)

In [23]:
data_2016['bedbathratio'] = np.where((data_2016['bedroomcnt']==0) | (data_2016['bathroomcnt']==0), 0,data_2016['bedbathratio'])
data_2017['bedbathratio'] = np.where((data_2017['bedroomcnt']==0) | (data_2017['bathroomcnt']==0), 0,data_2017['bedbathratio'])

In [21]:
data_2016['avgroomsize'] = np.where((data_2016['roomcnt']==0) , 0,data_2016['avgroomsize'])
data_2017['avgroomsize'] = np.where((data_2017['roomcnt']==0) , 0,data_2017['avgroomsize'])

In [24]:
data_2016.to_csv("C:/Users/Deepika Keswarap/Documents/projecta/all/featureengg/feature2016.csv", index=False, encoding='utf8')
data_2017.to_csv("C:/Users/Deepika Keswarap/Documents/projecta/all/featureengg/feature2017.csv", index=False, encoding='utf8')